In [2]:
import tensorflow as tf

import numpy as np
import os
import time

In [3]:
path_to_file = '../bible.txt'

In [4]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))

Length of text: 834418 characters


In [5]:
print(text[:500])

 Евангелие



 От Матфея святое благовествование


 Глава 1

  1  Родословие Иисуса Христа, Сына Давидова, Сына Авраамова.
  2  Авраам родил Исаака; Исаак родил Иакова; Иаков родил Иуду и братьев его;
  3  Иуда родил Фареса и Зару от Фамари; Фарес родил Есрома; Есром родил Арама;
  4  Арам родил Аминадава; Аминадав родил Наассона; Наассон родил Салмона;
  5  Салмон родил Вооза от Рахавы; Вооз родил Овида от Руфи; Овид родил Иессея;
  6  Иессей родил Давида царя; Давид царь родил Со


In [6]:
text = text + text

In [7]:
# The unique characters in the file
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

88 unique characters


In [8]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [9]:
text_as_int, len(text_as_int), len(text)

(array([ 2, 32, 56, ...,  0,  1,  0]), 1668836, 1668836)

### train and target

In [9]:
# The maximum length sentence you want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

 
Е
в
а
н


In [10]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

' Евангелие\r\n\r\n\r\n\r\n От Матфея святое благовествование\r\n\r\n\r\n Глава 1\r\n\r\n  1 \xa0Родословие Иисуса Христа, '
'Сына Давидова, Сына Авраамова.\r\n  2 \xa0Авраам родил Исаака; Исаак родил Иакова; Иаков родил Иуду и брат'
'ьев его;\r\n  3 \xa0Иуда родил Фареса и Зару от Фамари; Фарес родил Есрома; Есром родил Арама;\r\n  4 \xa0Арам '
'родил Аминадава; Аминадав родил Наассона; Наассон родил Салмона;\r\n  5 \xa0Салмон родил Вооза от Рахавы; '
'Вооз родил Овида от Руфи; Овид родил Иессея;\r\n  6 \xa0Иессей родил Давида царя; Давид царь родил Соломон'


In [11]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

Print the first example input and target values:

In [12]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  ' Евангелие\r\n\r\n\r\n\r\n От Матфея святое благовествование\r\n\r\n\r\n Глава 1\r\n\r\n  1 \xa0Родословие Иисуса Христа,'
Target data: 'Евангелие\r\n\r\n\r\n\r\n От Матфея святое благовествование\r\n\r\n\r\n Глава 1\r\n\r\n  1 \xa0Родословие Иисуса Христа, '


In [13]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [14]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 128

# Number of RNN units
rnn_units = 1024

In [15]:
model = tf.keras.Sequential(
    [
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
        tf.keras.layers.LSTM(rnn_units, return_sequences=True),
        tf.keras.layers.Dense(vocab_size)
    ]
)

In [16]:
class RNNgenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, batch_size):
        super(RNNgenerator, self).__init__()
        
        self.emb = tf.keras.layers.Embedding(vocab_size, embedding_dim)
                                 
        self.gru1 = tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            recurrent_initializer='glorot_uniform')
        self.gru2 = tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            recurrent_initializer='glorot_uniform')
                           
        self.fc = tf.keras.layers.Dense(vocab_size)

    def call(self, x):
        emb_x = self.emb(x)
        x1 = self.gru1(emb_x)
        x = x1
        for _ in range(3):
            x = self.gru2(x)
        #x = self.gru1(x)
        x = (x + x1)/2
        return self.fc(x)

model = RNNgenerator(vocab_size, embedding_dim, BATCH_SIZE)

In [17]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
                                 
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [18]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
                                 
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [19]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [20]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 88) # (batch_size, sequence_length, vocab_size)


In [21]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 128)         11264     
_________________________________________________________________
gru_2 (GRU)                  (None, None, 1024)        3545088   
_________________________________________________________________
gru_3 (GRU)                  (None, None, 1024)        6297600   
_________________________________________________________________
gru_4 (GRU)                  (None, None, 1024)        6297600   
_________________________________________________________________
dense_2 (Dense)              (None, None, 88)          90200     
Total params: 16,241,752
Trainable params: 16,241,752
Non-trainable params: 0
_________________________________________________________________


In [22]:
example_batch_predictions[0]

<tf.Tensor: shape=(100, 88), dtype=float32, numpy=
array([[ 3.8717419e-04,  4.9504539e-04, -4.9765961e-04, ...,
        -3.9090408e-04,  1.1322787e-03,  7.3167827e-04],
       [ 4.3125133e-04,  8.8875357e-04, -1.4990410e-03, ...,
         5.7589066e-05,  1.9211144e-03,  4.9542275e-04],
       [ 7.1455771e-04,  1.6016746e-03, -1.5993377e-03, ...,
         1.6167706e-03,  2.3126327e-03,  1.0426301e-03],
       ...,
       [ 2.7031756e-03, -9.6849422e-04, -4.5023235e-03, ...,
        -3.5595004e-03,  5.6592808e-03,  2.8479854e-03],
       [ 3.1707436e-03, -2.8883945e-04, -4.9850922e-03, ...,
        -3.0793624e-03,  6.0811089e-03,  3.2833894e-03],
       [ 2.9445298e-03, -7.9178018e-04, -5.8287946e-03, ...,
        -2.3779003e-03,  5.9041744e-03,  3.0803126e-03]], dtype=float32)>

In [23]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [24]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'н Человеческий идёт по предназначению, но горе тому человеку, которым Он предаётся.\r\n  23 \xa0И они нач'

Next Char Predictions: 
 '0\nМтХ9вНШх\xa0ъОвЧяею0Н\r\xa0?эХ«хщ,К(ВъУо2ая;б-л1ъЭ4ШФУЖъ ]\rЯа] ]ФщёнЯсмсэЯЧ]сгОШ!5рщлрСъпфжГр рьФВ96Ю;фХЗ'


## Train the model

In [25]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 88)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.477438


In [26]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

In [27]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    period=20,
    save_weights_only=True)

### Execute the training

In [28]:
EPOCHS = 20

In [29]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
258/258 [==============================] - 1076s 4s/step - loss: 2.5948
Epoch 2/20
258/258 [==============================] - 1145s 4s/step - loss: 1.5777
Epoch 3/20
258/258 [==============================] - 1149s 4s/step - loss: 1.2998
Epoch 4/20
258/258 [==============================] - 1081s 4s/step - loss: 1.1485
Epoch 5/20
258/258 [==============================] - 1086s 4s/step - loss: 1.0181
Epoch 6/20
258/258 [==============================] - 1002s 4s/step - loss: 0.8860
Epoch 7/20
258/258 [==============================] - 1001s 4s/step - loss: 0.7556
Epoch 8/20
258/258 [==============================] - 1000s 4s/step - loss: 0.6311
Epoch 9/20
258/258 [==============================] - 1004s 4s/step - loss: 0.5229
Epoch 10/20
258/258 [==============================] - 1003s 4s/step - loss: 0.4356
Epoch 11/20
258/258 [==============================] - 1003s 4s/step - loss: 0.3677
Epoch 12/20
258/258 [==============================] - 1004s 4s/step - loss: 0.3189
E

## Generate text

In [30]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_20'

In [31]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [32]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 128)         11264     
_________________________________________________________________
gru_5 (GRU)                  (None, None, 1024)        3545088   
_________________________________________________________________
gru_6 (GRU)                  (None, None, 1024)        6297600   
_________________________________________________________________
gru_7 (GRU)                  (None, None, 1024)        6297600   
_________________________________________________________________
dense_3 (Dense)              (None, None, 88)          90200     
Total params: 16,241,752
Trainable params: 16,241,752
Non-trainable params: 0
_________________________________________________________________


In [33]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 500

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 0.1

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [34]:
text_ = generate_text(model, start_string=u"И вот идет уже ")
print(text_)

И вот идет уже ни и и и и и и и и и и и и и и и во и и и ни и и и и и и и и и и и ни и и и и и и ни ни и и и и и и и и ни и и и и и и и и и и и и и ни и ни и и и и и во и и и во и и и и и и и и во и и пре и и и и и и и и и и и во и и и и и и во и и и и и и и и и и ни и и ни и и и и и пре и и и и и и и и и ни и и и и и и и и ни во и и и и ни и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и пре и и и и и и во и и и и и ни пре и и и и и и и сто и и и и и и и и и и и и и и и 
